In [ ]:
import torch
from torch import nn
from torch.nn import functional as F
import importlib
import sys
import nrrd
import numpy as np

sys.path.append('/mnt/raid/C1_ML_Analysis/source/famli-ultra-sim/')
sys.path.append('/mnt/raid/C1_ML_Analysis/source/famli-ultra-sim/dl/')
sys.path.append('/mnt/raid/C1_ML_Analysis/source/famli-ultra-sim/dl/nets')
from dl.nets import us_simu
import plotly.express as px
import pandas as pd

from loaders.mr_us_dataset import DiffusorSampleDataset
from transforms.mr_transforms import VolumeTrainTransforms, VolumeEvalTransforms

import os

In [ ]:
model = us_simu.USVQGanReconstruction.load_from_checkpoint('/mnt/raid/C1_ML_Analysis/train_output/simu_reconstruction/USVQGanReconstructionLogger/v0.1/epoch=57-val_loss_adv=0.16.ckpt', mount_point='/mnt/raid/C1_ML_Analysis')
model = model.eval().cuda()

In [ ]:
mount_point = '/mnt/raid/C1_ML_Analysis/'
df_train = pd.read_csv(os.path.join(mount_point, 'simulated_data_export/studies_merged/diffusor_resampled_train.csv'))
img_column = 'img_path'
transform = VolumeEvalTransforms()

train_ds = DiffusorSampleDataset(df_train, mount_point, img_column=img_column, transform=transform, num_samples=1000, return_ridx = True)
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=1)

In [ ]:
def rotate_3d_tensor_around_axis(tensor, angle, axis_vector):
    """
    Rotates a 3D tensor by a given angle around a specified axis vector.
    
    Parameters:
        tensor (torch.Tensor): The input tensor of shape (B, NC, D, W, H).
        angle (float): The rotation angle in degrees.
        axis_vector (torch.Tensor): The axis vector to rotate around of shape (3,).
        
    Returns:
        torch.Tensor: The rotated tensor.
    """
    B, NC, D, W, H = tensor.shape
    angle = torch.tensor(angle, dtype=torch.float32) * torch.pi / 180  # Convert angle to radians

    axis_vector = axis_vector / torch.norm(axis_vector)  # Normalize the axis vector
    u, v, w = axis_vector

    cos_angle = torch.cos(angle)
    sin_angle = torch.sin(angle)
    one_minus_cos = 1 - cos_angle

    # Rodrigues' rotation formula
    rotation_matrix = torch.tensor([
        [cos_angle + u**2 * one_minus_cos, u*v * one_minus_cos - w * sin_angle, u*w * one_minus_cos + v * sin_angle, 0],
        [v*u * one_minus_cos + w * sin_angle, cos_angle + v**2 * one_minus_cos, v*w * one_minus_cos - u * sin_angle, 0],
        [w*u * one_minus_cos - v * sin_angle, w*v * one_minus_cos + u * sin_angle, cos_angle + w**2 * one_minus_cos, 0],
        [0, 0, 0, 1]
    ])

    # Create affine grid
    grid = F.affine_grid(rotation_matrix[:3, :].unsqueeze(0).cuda(), tensor.size(), align_corners=False)
    rotated_tensor = F.grid_sample(tensor, grid, align_corners=False)
    
    return rotated_tensor

In [ ]:
features = []
ridxs = []
with torch.no_grad():
    for batch in train_dl:
        X, X_origin, X_end, ridx = batch
        X = rotate_3d_tensor_around_axis(X.float().cuda(), torch.rand(1).item() * 360, torch.rand(3).cuda())
        Y = model.volume_sampling(X.float().cuda(), X_origin.cuda(), X_end.cuda())
        features.append(model.vqvae.encoder(Y).flatten(start_dim=1,))
        ridxs.append(ridx)

In [ ]:

features = torch.cat(features, dim=0).cpu().numpy()
ridxs = torch.cat(ridxs, dim=0).cpu().numpy()
np.save('/mnt/raid/C1_ML_Analysis/train_output/simu_reconstruction/USVQGanReconstructionLogger/v0.1/encoded_features.npy', features)
np.save('/mnt/raid/C1_ML_Analysis/train_output/simu_reconstruction/USVQGanReconstructionLogger/v0.1/ridxs.npy', ridxs)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
features_t = pca.fit_transform(features)

In [ ]:
feat_df = pd.DataFrame(features_t, columns=['PCA1', 'PCA2'])
feat_df['ridx'] = ridxs

px.scatter(feat_df, x='PCA1', y='PCA2', color='ridx').show()